# Gradient Boost on Network Model Cohorts
Including Simplified Menu Type as a feature.

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import os
import time
import itertools
import collections
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import multiprocessing
import queue

import data_functions as dataf
import postgres_functions as pg
import plot_functions as plotter
from clustering_query import snowflake_query

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib

/Users/u1b1700/.pyenv/versions/3.7.2/envs/cohorts_venv/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
def add_smt(df):
    '''Gets customer info to add dummy columns for smplfd_menu_desc.

    :param df: dataframe with cust_nbr and div_nbr
    :type df: pd.DataFrame
    '''
    # pull cust_info to add smplfd_menu_desc, then dummy it
    cust_info = dataf.get_cust_info()
    df = pd.merge(df, cust_info.loc[:, ['cust_nbr', 'div_nbr',
                                        'smplfd_menu_desc']],
                  on=['cust_nbr', 'div_nbr'],
                  how='inner')

    df = pd.get_dummies(df,
                        prefix='',
                        prefix_sep='',
                        columns=['smplfd_menu_desc'],
                        dummy_na=False)
    
    return df

### Initialize Threshold and pkl name.


In [3]:
partial_pkl_name = 'ind_liu'
table_name = 'cohorts_first_pass'
pyr_seg = 'IND'
f1_threshold = 0.1
min_cohort_size = 75
start_time = time.time()

## Data
Get data from postgres table ```cohorts_first_pass```. Then query Snowflake for PIM Group Description to merge to dataframe.

In [15]:
# cohorts = pg.get_first_pass_table(table_name)
cohorts = pd.read_csv(f'../data/{table_name}_{pyr_seg}.csv')

In [16]:
cohorts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6912116 entries, 0 to 6912115
Data columns (total 10 columns):
cust_nbr                  int64
div_nbr                   int64
pim_grp_id_actl           int64
case_volume               int64
cust_grp_pct_of_basket    float64
source                    object
target                    object
source_cohort             object
target_cohort             object
last_update               object
dtypes: float64(1), int64(4), object(5)
memory usage: 527.4+ MB


In [18]:
# reduce dataframe to only max date
cohorts = cohorts[cohorts['last_update'] == cohorts['last_update'].max()]
cohorts['pim_grp_id_actl'] = cohorts['pim_grp_id_actl'].astype(str)
cohorts['cust_nbr'] = cohorts['cust_nbr'].astype(str)

In [19]:
prod_info = dataf.get_product_info()
cohorts = pd.merge(cohorts, prod_info, on='pim_grp_id_actl', how='inner')

Saved product data less than 1 week old.
Importing product data from prod_desc.csv...


In [20]:
cohorts.head()

,cust_nbr,div_nbr,pim_grp_id_actl,case_volume,cust_grp_pct_of_basket,source,target,source_cohort,target_cohort,last_update,pim_group_description,usf_category_description
0,10000560,2110,1370,45,0.036378,s100005602110,t1370,ind_cohort_0,ind_cohort_20,2019-05-20,"APPETIZERS, ONIONS, BREADED & BATTERED",APPETIZERS AND COATED VEGETABLES
1,1000074,2230,1370,14,0.013527,s10000742230,t1370,ind_cohort_2,ind_cohort_20,2019-05-20,"APPETIZERS, ONIONS, BREADED & BATTERED",APPETIZERS AND COATED VEGETABLES
2,1000710,2270,1370,279,0.063538,s10007102270,t1370,ind_cohort_6,ind_cohort_20,2019-05-20,"APPETIZERS, ONIONS, BREADED & BATTERED",APPETIZERS AND COATED VEGETABLES
3,1001338,2135,1370,2,0.000394,s10013382135,t1370,ind_cohort_1,ind_cohort_20,2019-05-20,"APPETIZERS, ONIONS, BREADED & BATTERED",APPETIZERS AND COATED VEGETABLES
4,1001726,2240,1370,27,0.002001,s10017262240,t1370,ind_cohort_8,ind_cohort_20,2019-05-20,"APPETIZERS, ONIONS, BREADED & BATTERED",APPETIZERS AND COATED VEGETABLES


## Data Aggregation and Customer-Group Crosstab
Create a dataframe that gives a crosstab with ```cust_nbr``` as index and ```pim_group_description``` as columns, and ```cust_grp_pct_of_basket``` as values.

In [21]:
cust_grp = dataf.create_crosstab(cohorts, 'pim_group_description', 'cust_grp_pct_of_basket')

-----------
Creating crosstab...


### Add Simplified Menu Type
Get customer info, then dummy Simplified Menu Type

In [22]:
cust_grp = add_smt(cust_grp)

Saved customer data less than 1 week old.
Importing customer data from cust_data.csv...


In [23]:
cust_grp.head()

,cust_nbr,div_nbr,"1000 ISLAND SALAD DRESSING, BULK, REFRIGERATED","1000 ISLAND SALAD DRESSING, BULK, SHELF STABLE","1000 ISLAND SALAD DRESSING, SINGLE SERVICE, REFRIGERATED","1000 ISLAND SALAD DRESSING, SINGLE SERVICE, SHELF STABLE",ADMINISTRATIVE,AIR FRESHENER,ALL PURPOSE CLEANER / POLISH,"ALLIGATOR, FROZEN",...,FROZEN DESSERTS,HAMBURGERS,LATIN AMERICAN,MEXICAN,OTHER ASIAN,OTHER ETHNIC FOOD,"PIZZA, PASTA & ITALIAN","SANDWICHES, SOUPS AND SALADS, DELI, SUBS & HOT DOG",SMT UNAVAILABLE,"STEAK, SEAFOOD & FISH"
0,10000560,2110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1000066,2230,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
2,1000074,2230,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,1000090,2230,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
4,10001238,2110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
# initialize columns to use as features of model
model_cols = cust_grp.columns[2:]

In [25]:
model_cols

Index(['1000 ISLAND SALAD DRESSING, BULK, REFRIGERATED',
       '1000 ISLAND SALAD DRESSING, BULK, SHELF STABLE',
       '1000 ISLAND SALAD DRESSING, SINGLE SERVICE, REFRIGERATED',
       '1000 ISLAND SALAD DRESSING, SINGLE SERVICE, SHELF STABLE',
       'ADMINISTRATIVE', 'AIR FRESHENER', 'ALL PURPOSE CLEANER / POLISH',
       'ALLIGATOR, FROZEN', 'ALMONDS', 'AMMONIA & BLEACH',
       ...
       'FROZEN DESSERTS', 'HAMBURGERS', 'LATIN AMERICAN', 'MEXICAN',
       'OTHER ASIAN', 'OTHER ETHNIC FOOD', 'PIZZA, PASTA & ITALIAN',
       'SANDWICHES, SOUPS AND SALADS, DELI, SUBS & HOT DOG', 'SMT UNAVAILABLE',
       'STEAK, SEAFOOD & FISH'],
      dtype='object', length=2218)

In [26]:
# # write out features to use with pickled model
# dataf.write_jsonl_file(model_cols, f'../data/model_features_{partial_pkl_name}.jsonl')

In [27]:
cust_grp.head()

,cust_nbr,div_nbr,"1000 ISLAND SALAD DRESSING, BULK, REFRIGERATED","1000 ISLAND SALAD DRESSING, BULK, SHELF STABLE","1000 ISLAND SALAD DRESSING, SINGLE SERVICE, REFRIGERATED","1000 ISLAND SALAD DRESSING, SINGLE SERVICE, SHELF STABLE",ADMINISTRATIVE,AIR FRESHENER,ALL PURPOSE CLEANER / POLISH,"ALLIGATOR, FROZEN",...,FROZEN DESSERTS,HAMBURGERS,LATIN AMERICAN,MEXICAN,OTHER ASIAN,OTHER ETHNIC FOOD,"PIZZA, PASTA & ITALIAN","SANDWICHES, SOUPS AND SALADS, DELI, SUBS & HOT DOG",SMT UNAVAILABLE,"STEAK, SEAFOOD & FISH"
0,10000560,2110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1000066,2230,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
2,1000074,2230,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,1000090,2230,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
4,10001238,2110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


### Merge Crosstab with Cohort Assignments

In [28]:
cohort_unique_df = cohorts.loc[:, ['cust_nbr', 'div_nbr', 'source_cohort']]
cohort_unique_df = cohort_unique_df.drop_duplicates()

# merge crosstab with cust_df
cust_grp = pd.merge(cust_grp, cohort_unique_df, on = ['cust_nbr', 'div_nbr'], how = 'inner')

### Write File out for DataRobot

In [33]:
# drop columns not needed for input
outfile = cust_grp.copy()
outfile.drop('cust_nbr', axis=1, inplace=True)
outfile.drop('div_nbr', axis=1, inplace=True)

# write out file for datarobot
outfile.to_csv(f'../data/dr_{partial_pkl_name}.csv', index=False)

### Get Cohort Dummies and Initialize labelCols

In [29]:
cust_grp.head()

,cust_nbr,div_nbr,"1000 ISLAND SALAD DRESSING, BULK, REFRIGERATED","1000 ISLAND SALAD DRESSING, BULK, SHELF STABLE","1000 ISLAND SALAD DRESSING, SINGLE SERVICE, REFRIGERATED","1000 ISLAND SALAD DRESSING, SINGLE SERVICE, SHELF STABLE",ADMINISTRATIVE,AIR FRESHENER,ALL PURPOSE CLEANER / POLISH,"ALLIGATOR, FROZEN",...,HAMBURGERS,LATIN AMERICAN,MEXICAN,OTHER ASIAN,OTHER ETHNIC FOOD,"PIZZA, PASTA & ITALIAN","SANDWICHES, SOUPS AND SALADS, DELI, SUBS & HOT DOG",SMT UNAVAILABLE,"STEAK, SEAFOOD & FISH",source_cohort
0,10000560,2110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,ind_cohort_0
1,1000066,2230,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,ind_cohort_1
2,1000074,2230,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,ind_cohort_2
3,1000090,2230,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,ind_cohort_3
4,10001238,2110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,ind_cohort_4


In [30]:
# get dummies for cohort columns
cust_grp = pd.get_dummies(cust_grp, prefix='', prefix_sep='', columns=['source_cohort'], dummy_na=False)

# initialize label_cols to iterate through models (ignore feat columns, cust_nbr, div_nbr).
label_cols = cust_grp.columns[len(model_cols) + 2:]

Merge additional customer info for output.

In [31]:
# get cust_corp info
query = '''
        SELECT cust_nbr, 
            div_nbr, 
            cust_nm, 
            smplfd_menu_desc
        FROM cust_corp
        '''
cust_names = snowflake_query(query)

# datassentials info
datass = dataf.get_datassentials_custs()

------------------
Submitting Snowflake query...
-----------
Retriving Datassentials customer info...
------------------
Submitting Snowflake query...


In [32]:
# merge
cust_grp = pd.merge(cust_grp, cust_names, on = ['cust_nbr', 'div_nbr'], how = 'left')
cust_grp = pd.merge(cust_grp, datass, on = ['cust_nbr', 'div_nbr'], how = 'left')

## Gradient Boost for Feature Importance
We want to look at the feature importance that comes from a Gradient Boost model for each of the cohort labels. For each dummy column, we use a One-vs-Rest (OvR) schema to train the model for that cohort.

At each iteration the classes are imbalanced, so we are using a stratified train-test split for cross-validation. Consider upsampling?

In [38]:
# # remove previous pkl files
# print('--------\nRemoving old pkl files...')
# for filename in os.listdir('pkl'):
#     if partial_pkl_name in filename:
#         print(filename)
#         os.remove(f'pkl/{filename}')
        
# print('Time elapased, in minutesd: {:.2f}'.format((time.time() - start_time) / 60))

ind_liu_ind_cohort_1.pkl
ind_liu_ind_cohort_0.pkl


In [79]:
def train_and_pickle_model(df, X, y, label_col, pq):
#     print(f'\n+++++++++++++++++++++\n Label: {label_col}')
    y = df.loc[:, label_col]

    print(f'Label Value Counts:\n{y.value_counts()}')

    # if there are fewer than 75 customers in a cohort generated by the network model, don't pickle
    if y.sum() < min_cohort_size:
        print('Cohort size below threshold.')

    else:
        # perform a stratified train-test split for imbalanced classes
        X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=123)

        gb = GradientBoostingClassifier(loss='deviance', 
                                        learning_rate = 0.05, 
                                        n_estimators=2500, 
                                        criterion='friedman_mse', 
                                        min_samples_split=2, 
                                        min_samples_leaf=1, 
                                        max_depth=5, 
                                        random_state=123, 
                                        max_features=0.2, 
                                        max_leaf_nodes=None, 
                                        subsample=1.0,
                                        verbose=0)

        gb.fit(X_train, y_train)
        y_pred = gb.predict(X_test)

        # get scores
        f1, recall, precision, accuracy = dataf.score_model(y_test, y_pred)
        cohort_scores = {'cohort': label_col, 
                   'scores': {'f1': f1, 
                              'recall': recall, 
                              'precision': precision, 
                              'accuracy': accuracy}}
        dataf.write_json_file(cohort_scores, '../data/model_scores_{partial_pkl_name}.jsonl')
        print(f'{label_col}, f1: {f1}')

        # Plot the feature importance
        feat_scores = pd.Series(gb.feature_importances_, index=model_cols)

        top_scores = feat_scores.sort_values()[::-1][:20][::-1]

        # plot feature importances
        ax = top_scores.plot(kind='barh', figsize=(10, 8), color='g')
        ax.set_title(f'Feature Importance (Friedman MSE) for {label_col}')
        ax.set_xlabel('Contribution to Information Gain')
        plt.show()

        # print out top 5 smplfd_menu_desc and cuisine_secondary
        subdf = df[df[label_col] == 1]
        for col in ['smplfd_menu_desc', 'cuisine_secondary']:
            # create ordered dictionary of value_counts and get top
            rankings = collections.OrderedDict(subdf[col].value_counts())
            tops = itertools.islice(rankings.items(), 0, 5)
            plotter.plot_dict(dict(tops), f'Top 5 {col} for {label_col}', figsize=(8, 4))

        # print out top 25 cust_nm
        rankings = collections.OrderedDict(subdf['cust_nm'].value_counts())
        tops = itertools.islice(rankings.items(), 0, 25)
        print(f'--------------\nTop 25 cust_nm for {label_col}\n--------------')
        for key, value in tops:
            print(f'{key}\t{value}')

        if f1 < f1_threshold:
            print('F1 Score below threshold! Not pickling this model.\n-------')
            
        else:
            # append scores to lists
#             f1_list.append(f1)
#             recall_list.append(recall)
#             precision_list.append(precision)
#             accuracy_list.append(accuracy)
            
            pq.put(f1)
#             proc_queue.put(recall)
#             proc_queue.put(precision)
#             proc_queue.put(accuracy)

            # append top features to outfile
#             outfile = outfile.append(dataf.feat_importance_df(top_scores, label_col))
#             proc_queue.put(dataf.feat_importance_df(top_scores, label_col))
            
#             # pickle the model
#             print(f'Pickling {label_col} model...')
#             pickle_path = f'pkl/{partial_pkl_name}_{label_col}.pkl'
#             joblib.dump(gb, pickle_path)
#             model_list.append(pickle_path[4:-4])
#             print('Pickling complete.')
            
    print('Time elapsed, in minutes: {:.2f}'.format((time.time() - start_time) / 60))
        

In [80]:
proc_queue = multiprocessing.Queue()
label_list = list(label_cols.copy())

# # initialize lists for models and mean scores
model_list = []
f1_list = []
recall_list = []
precision_list = []
accuracy_list = []
feat_df_list = []

# initialize dataframe to write out feature importances
outfile = pd.DataFrame()

X = cust_grp.loc[:, model_cols]

while len(label_list) > 0:
    sublist = []
    for lab in label_list[:4]:
        sublist.append(lab)
        label_list.remove(lab)
        
    print(sublist)
    
    jobs = []
    for label_col in sublist:
        y = cust_grp.loc[:, label_col]
        
        jobs.append(multiprocessing.Process(target=train_and_pickle_model, 
                                            args=(cust_grp, X, y, label_col, 
                                                  proc_queue)))
                    
    for job in jobs:
        job.start()
        
    for job in jobs:
        job.join()
        f1_list.append(proc_queue.get())
#         recall_list.append(proc_queue.get())
#         precision_list.append(proc_queue.get())
#         accuracy_list.append(proc_queue.get())
#         feat_df_list.append(proc_queue.get())

        


['ind_cohort_0', 'ind_cohort_1', 'ind_cohort_10', 'ind_cohort_11']
Label Value Counts:
0    76969
1     5598
Name: ind_cohort_0, dtype: int64Label Value Counts:
0    77566
1     5001
Name: ind_cohort_1, dtype: int64

Label Value Counts:
0    78375
1     4192
Name: ind_cohort_10, dtype: int64
Label Value Counts:
0    81889
1      678
Name: ind_cohort_11, dtype: int64


KeyboardInterrupt: 

Process Process-67:
Traceback (most recent call last):
  File "/Users/u1b1700/.pyenv/versions/3.7.2/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/u1b1700/.pyenv/versions/3.7.2/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-79-e192e81d905d>", line 28, in train_and_pickle_model
    gb.fit(X_train, y_train)
  File "/Users/u1b1700/.pyenv/versions/3.7.2/envs/cohorts_venv/lib/python3.7/site-packages/sklearn/ensemble/gradient_boosting.py", line 1465, in fit
    begin_at_stage, monitor, X_idx_sorted)
  File "/Users/u1b1700/.pyenv/versions/3.7.2/envs/cohorts_venv/lib/python3.7/site-packages/sklearn/ensemble/gradient_boosting.py", line 1529, in _fit_stages
    X_csc, X_csr)
  File "/Users/u1b1700/.pyenv/versions/3.7.2/envs/cohorts_venv/lib/python3.7/site-packages/sklearn/ensemble/gradient_boosting.py", line 1194, in _fit_stage
    check_input=False, X_idx_sorted=X_i

In [47]:
# print mean scores
print('-----------\nMean Scores:')
print(f'\tF1 Score: {np.mean(f1_list)}')
print(f'\tRecall Score: {np.mean(recall_list)}')
print(f'\tPrecision Score: {np.mean(precision_list)}')
print(f'\tAccuracy Score: {np.mean(accuracy_list)}')

-----------
Mean Scores:


TypeError: unsupported operand type(s) for /: 'ListProxy' and 'int'

In [ ]:
# # # write out files for feature importance and f1 scores
# outfile.to_csv(f'../data/feat_imp_{partial_pkl_name}.csv', index=False)

# f1_df = pd.DataFrame({'pkl_name': model_list, 'f1_score': f1_list})
# f1_df.to_csv(f'../data/f1_scores_{partial_pkl_name}.csv', index=False)